In [1]:
import pandas as pd

# Load event data (already exported from StatsBomb)
events = pd.read_csv('../data/euro2024_events.csv')

# Extract shots dataframe
shots_euro = events[events['type'] == 'Shot'].copy()
euro_matches = pd.read_csv('../data/euro2024_matches.csv')

matches = euro_matches[['match_id', 'home_team', 'away_team']]
def get_opponent(row):
    if row['team'] == row['home_team']:
        return row['away_team']
    elif row['team'] == row['away_team']:
        return row['home_team']
    return None

shots_euro = shots_euro.merge(matches, on='match_id', how='left')
shots_euro['opponent_team'] = shots_euro.apply(get_opponent, axis=1)

pd.set_option('display.max_columns', None)

C:\Users\pavle\AppData\Local\Temp\ipykernel_5772\727380205.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,44,45,49,50,51,52,54,55,56,57,58,59,60,61,62,64,65,66,67,68,70,71,72,73,74,75,80,87,88,89,90,91,92,93,94,95,96,97,98,99,100,102,103,104,106,108) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv('../data/euro2024_events.csv')


In [2]:
import pandas as pd
import ast

fifty_fifties = pd.read_csv('../data/euro2024_50_50s.csv')

# Parse the JSON-like strings (they use single quotes) into real dicts
fifty_fifties['50_50_parsed'] = fifty_fifties['50_50'].apply(
    lambda s: ast.literal_eval(s) if pd.notna(s) and isinstance(s, str) else None
)

# Extract outcome fields for easy filtering
fifty_fifties['outcome'] = fifty_fifties['50_50_parsed'].apply(lambda d: d.get('outcome') if isinstance(d, dict) else None)
fifty_fifties['outcome_id'] = fifty_fifties['outcome'].apply(lambda o: o.get('id') if isinstance(o, dict) else None)
fifty_fifties['outcome_name'] = fifty_fifties['outcome'].apply(lambda o: o.get('name') if isinstance(o, dict) else None)

# Filter successful 50/50s (choose by name or by id depending on your data)
success_names = ['Won', 'Success To Team'] 
success_ids = [108, 147]  # adjust if your CSV uses different ids , 109, 148
unsuccess_names = ['Lost', 'Failure To Team']
unsuccess_ids = [109, 148]  # adjust if your CSV uses different ids

successful = fifty_fifties[
    fifty_fifties['outcome_name'].isin(success_names) |
    fifty_fifties['outcome_id'].isin(success_ids)
]

unsuccessful = fifty_fifties[
    fifty_fifties['outcome_name'].isin(unsuccess_names) |
    fifty_fifties['outcome_id'].isin(unsuccess_ids)
]

print(fifty_fifties['50_50_parsed'].shape)
print(successful.shape)
print(unsuccessful.shape)

print(successful[['50_50_parsed', 'outcome_id', 'outcome_name']].head())
print(unsuccessful[['50_50_parsed', 'outcome_id', 'outcome_name']].head())

# Count occurrences of each outcome
outcome_counts = fifty_fifties['outcome_name'].value_counts()
print(outcome_counts)

(293,)
(114, 28)
(147, 28)
                                        50_50_parsed  outcome_id  \
1  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   
3  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   
5  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   
7              {'outcome': {'id': 4, 'name': 'Won'}}           4   
9  {'outcome': {'id': 3, 'name': 'Success To Team'}}           3   

      outcome_name  
1  Success To Team  
3  Success To Team  
5  Success To Team  
7              Won  
9  Success To Team  
                             50_50_parsed  outcome_id outcome_name
0  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
2  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
4  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
6  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
8  {'outcome': {'id': 1, 'name': 'Lost'}}           1         Lost
outcome_name
Lost                   

In [3]:
fifty_fifties['50_50_parsed']

0                 {'outcome': {'id': 1, 'name': 'Lost'}}
1      {'outcome': {'id': 3, 'name': 'Success To Team'}}
2                 {'outcome': {'id': 1, 'name': 'Lost'}}
3      {'outcome': {'id': 3, 'name': 'Success To Team'}}
4                 {'outcome': {'id': 1, 'name': 'Lost'}}
                             ...                        
288                {'outcome': {'id': 4, 'name': 'Won'}}
289               {'outcome': {'id': 1, 'name': 'Lost'}}
290    {'outcome': {'id': 3, 'name': 'Success To Team'}}
291               {'outcome': {'id': 1, 'name': 'Lost'}}
292                {'outcome': {'id': 4, 'name': 'Won'}}
Name: 50_50_parsed, Length: 293, dtype: object

In [4]:
matches = pd.read_csv('../data/euro2024_matches.csv')

matches.head(5)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3942819,2024-07-10,22:00:00.000,Europe - UEFA Euro,2024,Netherlands,England,1,2,available,available,2024-07-15T07:54:23.583297,2024-07-15T07:57:36.684453,6,Semi-finals,Signal-Iduna-Park,Felix Zwayer,Ronald Koeman,Gareth Southgate,1.1.0,2,2
1,3943043,2024-07-14,22:00:00.000,Europe - UEFA Euro,2024,Spain,England,2,1,available,available,2024-07-15T15:48:50.315500,2024-07-15T15:52:24.778809,7,Final,Olympiastadion Berlin,François Letexier,Luis de la Fuente Castillo,Gareth Southgate,1.1.0,2,2
2,3942752,2024-07-09,22:00:00.000,Europe - UEFA Euro,2024,Spain,France,2,1,available,available,2024-07-10T08:51:07.032303,2024-07-10T13:01:26.597509,6,Semi-finals,Allianz Arena,Slavko Vinčić,Luis de la Fuente Castillo,Didier Deschamps,1.1.0,2,2
3,3942382,2024-07-06,22:00:00.000,Europe - UEFA Euro,2024,Netherlands,Turkey,2,1,available,available,2024-07-10T06:50:04.068615,2024-07-10T06:56:35.146850,5,Quarter-finals,Olympiastadion Berlin,Clément Turpin,Ronald Koeman,Vincenzo Montella,1.1.0,2,2
4,3942349,2024-07-05,22:00:00.000,Europe - UEFA Euro,2024,Portugal,France,0,0,available,available,2024-07-12T01:57:15.210934,2024-07-12T02:00:16.871522,5,Quarter-finals,Volksparkstadion,Michael Oliver,Roberto Martínez Montoliú,Didier Deschamps,1.1.0,2,2


In [6]:
fifty_fifties['period'].value_counts()

period
1    173
2    116
4      2
3      2
Name: count, dtype: int64